[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/petro1eum/trust_chain/blob/main/examples/trustchain_llm.ipynb)

# TrustChain + LLM Integration

This notebook demonstrates how to integrate TrustChain with popular LLM providers:

- **OpenAI** (GPT-4, GPT-3.5)
- **Anthropic** (Claude 3)
- **OpenRouter** (100+ models)
- **LangChain** (Tools & Agents)

Every tool call is cryptographically signed, creating an audit trail.

In [ ]:
# Install dependencies (required in Colab)
!pip install -q trustchain

In [ ]:
from dataclasses import dataclass
from typing import Any, Callable

from trustchain import TrustChain

## 1. Verified Tool Executor

This is the core pattern: wrap any tool with TrustChain signatures.

In [ ]:
@dataclass
class VerifiedToolResult:
    """Result of a verified tool execution."""
    tool_id: str
    result: Any
    signature: str
    verified: bool = True


class VerifiedToolExecutor:
    """Wraps tool execution with TrustChain signatures."""

    def __init__(self):
        self.tc = TrustChain()
        self.chain = []  # Signed responses
        self.tools = {}  # Registered tools

    def register_tool(self, name: str, func: Callable) -> None:
        """Register a tool function."""
        self.tools[name] = func

    def execute(self, tool_name: str, args: dict, metadata: dict = None) -> VerifiedToolResult:
        """Execute a tool and sign the result."""
        if tool_name not in self.tools:
            raise ValueError(f"Unknown tool: {tool_name}")

        # Execute the tool
        result = self.tools[tool_name](**args)

        # Get parent signature for chain of trust
        parent_sig = self.chain[-1].signature if self.chain else None

        # Sign the response
        signed = self.tc._signer.sign(
            tool_id=tool_name,
            data={"result": result, "metadata": metadata or {}},
            parent_signature=parent_sig,
        )

        self.chain.append(signed)

        return VerifiedToolResult(
            tool_id=tool_name,
            result=result,
            signature=signed.signature,
        )

    def verify_chain(self) -> bool:
        """Verify all responses in the chain."""
        for response in self.chain:
            if not self.tc.verify(response):
                return False
        return True

    def get_audit_trail(self) -> list:
        """Get audit trail for compliance."""
        return [
            {
                "tool_id": r.tool_id,
                "timestamp": r.timestamp,
                "signature": r.signature[:32] + "...",
            }
            for r in self.chain
        ]

## 2. Demo: Verified Agent Workflow

Simulate an AI agent making tool calls with cryptographic proofs.

In [ ]:
# Create executor
executor = VerifiedToolExecutor()

# Register mock tools (replace with real implementations)
executor.register_tool("search", lambda query: f"Results for: {query}")
executor.register_tool("calculate", lambda expr: str(eval(expr)))
executor.register_tool("summarize", lambda text: f"Summary: {text[:50]}...")

print("Registered tools:", list(executor.tools.keys()))

In [ ]:
# Simulate agent workflow
print("Agent executing verified tool calls...\n")

# Step 1: Calculate
r1 = executor.execute("calculate", {"expr": "2 + 2 * 10"})
print(f"1. calculate('2 + 2 * 10') = {r1.result}")
print(f"   Signature: {r1.signature[:40]}...")

# Step 2: Search (chained to previous)
r2 = executor.execute("search", {"query": f"number {r1.result}"})
print(f"\n2. search('number {r1.result}') = {r2.result}")
print(f"   Signature: {r2.signature[:40]}...")

# Step 3: Summarize (chained to previous)
r3 = executor.execute("summarize", {"text": r2.result})
print(f"\n3. summarize(...) = {r3.result}")
print(f"   Signature: {r3.signature[:40]}...")

In [ ]:
# Verify the entire chain
print("Verifying execution chain...")
is_valid = executor.verify_chain()
print(f"Chain valid: {is_valid}")
print(f"Chain length: {len(executor.chain)} tool calls")

In [ ]:
# Export audit trail
print("Audit Trail for Compliance:\n")
for i, entry in enumerate(executor.get_audit_trail(), 1):
    print(f"{i}. {entry['tool_id']}")
    print(f"   Timestamp: {entry['timestamp']}")
    print(f"   Signature: {entry['signature']}")

## 3. OpenAI Integration Pattern

Use with OpenAI function calling (GPT-4).

In [ ]:
# Simulate OpenAI function calling response
def simulate_openai_response():
    """Simulates what OpenAI would return for a tool call."""
    return {
        'model': 'gpt-4',
        'tool_calls': [
            {'id': 'call_abc123', 'function': {'name': 'get_weather', 'arguments': '{"location": "Tokyo"}'}}
        ]
    }

# Create executor and register tools
openai_executor = VerifiedToolExecutor()
openai_executor.register_tool('get_weather', lambda location: f"Sunny, 22°C in {location}")

# Simulate API response
api_response = simulate_openai_response()
print(f"Model: {api_response['model']}")

# Execute and sign each tool call
import json as json_module

for tc in api_response['tool_calls']:
    args = json_module.loads(tc['function']['arguments'])
    result = openai_executor.execute(
        tc['function']['name'],
        args,
        metadata={'model': api_response['model'], 'openai_call_id': tc['id']}
    )
    print(f"\nTool: {tc['function']['name']}")
    print(f"Args: {args}")
    print(f"Result: {result.result}")
    print(f"Signature: {result.signature[:40]}...")
    print(f"Verified: {openai_executor.tc.verify(openai_executor.chain[-1])}")

## 4. OpenRouter Integration

Use with any model via OpenRouter (Claude, Llama, Mistral, etc.).

In [ ]:
# Simulate OpenRouter response (no API key needed for demo)
def simulate_openrouter_response():
    """Simulates what OpenRouter would return for a tool call."""
    return {
        'model': 'anthropic/claude-3-sonnet',
        'tool_calls': [
            {'id': 'call_xyz', 'function': {'name': 'calculate', 'arguments': '{"expr": "15 * 7"}'}}
        ]
    }

# Process with TrustChain
executor = VerifiedToolExecutor()
executor.register_tool('calculate', lambda expr: str(eval(expr)))

# Simulate API response
api_response = simulate_openrouter_response()
print(f"Model: {api_response['model']}")

# Execute and sign each tool call
import json as json_module

for tc in api_response['tool_calls']:
    args = json_module.loads(tc['function']['arguments'])
    result = executor.execute(
        tc['function']['name'],
        args,
        metadata={'model': api_response['model'], 'call_id': tc['id']}
    )
    print(f"\nTool: {tc['function']['name']}")
    print(f"Args: {args}")
    print(f"Result: {result.result}")
    print(f"Signature: {result.signature[:40]}...")
    print(f"Verified: {executor.tc.verify(executor.chain[-1])}")

## 5. Key Takeaways

| Feature | Benefit |
|---------|----------|
| Ed25519 signatures | Cryptographic proof of tool execution |
| Chain of trust | Link all tool calls in a session |
| Audit trail | Export for compliance (SOC2, HIPAA) |
| Model-agnostic | Works with any LLM provider |

---

**Next steps:**
- [GitHub](https://github.com/petro1eum/trust_chain)
- [Documentation](https://github.com/petro1eum/trust_chain/wiki)
- [PyPI](https://pypi.org/project/trustchain/)